#Use a proper tensorflow version
Our model was build on TF2.14 and it does cause some compatibility issues with TF2.17 so make sure you use the right version

In [1]:
#Check tf version
import tensorflow as tf
#print(tf.__version__)

#The following force you to use tensorflow 2.14 but you need to restart. You may comment it if you know you are using 2.14
##Make sure you down grade!! otherwise, some part will fail due to compatibility.
#!pip3 install --upgrade tensorflow==2.14.0
print(tf.__version__)

2.13.0


#Get the dataset

In [4]:
# mount google drive on your runtime using and authorization code.
# more details here: https://colab.research.google.com/notebooks/io.ipynb
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
#Set directories correctly

In [6]:
import os
# Path to the directory containing the project files (CHANGE THIS PATH TO THE DIRECTORY ON YOUR COMPUTER)
PROJECT_ROOT_DIR = os.getcwd() + '/'

# Path to the directory containing the dataset relative to project file
DATA_DIR = './GTSRB_dataset(2)/GTSRB_dataset/'

#path to the directory you want to use for saving models relative to the project file
MODEL_DIR = './'

In [7]:
# Funciton for loading the dataset
# Code from advml-traffic-sign (https://github.com/inspire-group/advml-traffic-sign)
def load_dataset_GTSRB(n_channel=3, train_file_name=None):
    """
    Load GTSRB data as a (datasize) x (channels) x (height) x (width) numpy
    matrix. Each pixel is rescaled to the range [0,1].
    """

    def load_pickled_data(file, columns):
        """
        Loads pickled training and test data.

        Parameters
        ----------
        file    : string
                          Name of the pickle file.
        columns : list of strings
                          List of columns in pickled data we're interested in.

        Returns
        -------
        A tuple of datasets for given columns.
        """

        with open(file, mode='rb') as f:
            dataset = pickle.load(f)
        return tuple(map(lambda c: dataset[c], columns))

    def preprocess(x, n_channel):
        """
        Preprocess dataset: turn images into grayscale if specified, normalize
        input space to [0,1], reshape array to appropriate shape for NN model
        """

        if n_channel == 3:
            # Scale features to be in [0, 1]
            x = (x / 255.).astype(np.float32)
        else:
            # Convert to grayscale, e.g. single Y channel
            x = 0.299 * x[:, :, :, 0] + 0.587 * x[:, :, :, 1] + \
                0.114 * x[:, :, :, 2]
            # Scale features to be in [0, 1]
            x = (x / 255.).astype(np.float32)
            x = x[:, :, :, np.newaxis]
        return x

    # Load pickle dataset
    if train_file_name is None:
        x_train, y_train = load_pickled_data(
            PROJECT_ROOT_DIR + DATA_DIR + 'train.p', ['features', 'labels'])
    else:
        x_train, y_train = load_pickled_data(
            PROJECT_ROOT_DIR + DATA_DIR + train_file_name, ['features', 'labels'])
    x_val, y_val = load_pickled_data(
        PROJECT_ROOT_DIR + DATA_DIR + 'valid.p', ['features', 'labels'])
    x_test, y_test = load_pickled_data(
        PROJECT_ROOT_DIR + DATA_DIR + 'test.p', ['features', 'labels'])

    # Preprocess loaded data
    x_train = preprocess(x_train, n_channel)
    x_val = preprocess(x_val, n_channel)
    x_test = preprocess(x_test, n_channel)
    return x_train, y_train, x_val, y_val, x_test, y_test

In [8]:
import pickle
import numpy as np
# Load the images and labels. These images are RGB so we have 3 channels
imgs_train, labels_train, imgs_val, labels_val, imgs_test, labels_test = load_dataset_GTSRB(n_channel=3)

#Load the built model from Assignment 1 (Already on blackboard)

In [10]:
## Load the saved the model
from tensorflow.keras.models import load_model



#load our already trained model
# Your earlier directories should change to match this one
#You should be able to load the model once trained. The following line is used for that.
model = load_model(MODEL_DIR +'VGG_best(1).h5')

#Some constants that you might need
It is not neccessary that you will use all

In [11]:
# Set constants (GTSRB)
NUM_LABELS = 43                             # Number of labels or classes for classification
BATCH_SIZE = 128                            # Size of batch
HEIGHT = 32                                 # Height of input image
WIDTH = 32                                  # Width of input image
N_CHANNEL = 3                               # Number of channels
OUTPUT_DIM = 43                             # Number of output dimension

# Set training hyperparameters
NUM_EPOCH = 50                            # Number of epoch to train
LR = 0.0002                                 # Learning rate
RBW = True #restore best weights
PATIENCE = 5# how many epochs between improvements
sample_size = 100
alpha = 0.001
INPUT_SHAPE = (HEIGHT, WIDTH, N_CHANNEL)  # Input shape of model
IMG_SHAPE = (HEIGHT, WIDTH, N_CHANNEL)

# setting up labels

In [12]:

from tensorflow.keras.utils import to_categorical

#Convert the labels to one-hot encoding (to input to the models)

#Use the following sub-test dataset for your certified training
imgs_sub_test = imgs_test[0:500,:,:,:].copy()
labels_adv = labels_test[0:500].copy()




labels_train_cat = to_categorical(labels_train, NUM_LABELS)
labels_test_cat = to_categorical(labels_test, NUM_LABELS)
labels_val_cat = to_categorical(labels_val, NUM_LABELS)

#for testing adversarial inputs
labels_adv_cat = to_categorical(labels_adv,NUM_LABELS)

print('Labels train shape: {}'.format(labels_train.shape))
print('Labels train catagorical shape: {}\n'.format(labels_train_cat.shape))
print('Labels Adver shape: {}'.format(labels_adv.shape))
print('Labels Adver catagorical shape: {}'.format(labels_adv_cat.shape))

print('Imgs Adver shape: {}'.format(imgs_sub_test.shape))

Labels train shape: (34799,)
Labels train catagorical shape: (34799, 43)

Labels Adver shape: (500,)
Labels Adver catagorical shape: (500, 43)
Imgs Adver shape: (500, 32, 32, 3)


#Install ART if needed

In [13]:
#Install ART

#Some imports.
You might need more

In [14]:
from art.estimators.classification import TensorFlowV2Classifier

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

##Task 3
Let's do another privacy attack. This time, it is a “shadow models” attack, discussed in class. [This notebook](https://github.com/Trusted-AI/adversarial-robustness-toolbox/blob/main/notebooks/attack_membership_inference_shadow_models.ipynb) will be your guide. You are going to use ART again, and here are a few steps to follow:

1-	Build the shadow models based on the numbers provided below. Set the training ratio for them to 0.5.

2-	Measure the accuracies for them (use validation accuracy as a measure)

3-	Build the attack model. You can build any binary classifier. You are suggested to use a random forest classifier for simplicity.

4-	Evaluate your attack models for members and non-members (the evaluation data is already given to you).

5-	Calculate the precision and recall of your attack model.

Your changing parameter in this task is the number of shadow models. You will set it to 1,2 and 5. Please note the needed metrics so you can plan your code accordingly.

#Note on the implementation:
Our implementation will look strange. Shadow models will require lots and lots of data. Thus, in a general setup, researchers use 80% of the data for shadow models, and only 20% is for training/testing/validation. However, given that we are using a model already built from Assignment 1, we cannot use this general setup. Thus, our shadow models' data will concatenate training and testing while the validation will be left out to test performance accuracy. To evaluate the attack, we generated a separate dataset that is taken from training and testing. You will need to locate and work with that data. In general, your attack results won't be good due to the above design, but the idea is that you should analyze it.


# Data for evaluating the attack.

In [16]:
# Data for evaluating the attack.
from numpy.random import choice
n_sample_train = 3000
eval_data_idx=choice(len(imgs_train), n_sample_train)
members_samples, members_labels= imgs_train[eval_data_idx], labels_train_cat[eval_data_idx]
n_sample_train = 3000
eval_data_idx=choice(len(imgs_test), n_sample_train)
non_members_samples, non_members_labels= imgs_test[eval_data_idx], labels_test_cat[eval_data_idx]

#Impliment your attack based on the above steps

In [63]:
def calc_precision_recall(predicted, actual, positive_value=1):
    score = 0  # both predicted and actual are positive
    num_positive_predicted = 0  # predicted positive
    num_positive_actual = 0  # actual positive
    for i in range(len(predicted)):
        if predicted[i] == positive_value:
            num_positive_predicted += 1
        if actual[i] == positive_value:
            num_positive_actual += 1
        if predicted[i] == actual[i]:
            if predicted[i] == positive_value:
                score += 1

    if num_positive_predicted == 0:
        precision = 1
    else:
        precision = score / num_positive_predicted  # the fraction of predicted “Yes” responses that are correct
    if num_positive_actual == 0:
        recall = 1
    else:
        recall = score / num_positive_actual  # the fraction of “Yes” responses that are predicted correctly

    return precision, recall

In [80]:
print(imgs_train.shape,labels_train.shape)

(34799, 32, 32, 3) (34799,)


In [81]:
'''
x_target_train = members_samples
y_target_train = members_labels
x_target_test = non_members_samples
y_target_test = non_members_labels
x_target_train= x_target_train.reshape(x_target_train.shape[0], -1)
y_target_train = np.argmax(y_target_train, axis=1)
x_target_test= x_target_test.reshape(x_target_test.shape[0], -1)
y_target_test= np.argmax(y_target_test, axis=1)
'''
x_target_train = imgs_train
y_target_train = labels_train
x_target_test = imgs_test
y_target_test = labels_test
x_target_train= x_target_train.reshape(x_target_train.shape[0], -1)
#y_target_train = np.argmax(y_target_train, axis=1)
x_target_test= x_target_test.reshape(x_target_test.shape[0], -1)
#y_target_test= np.argmax(y_target_test, axis=1)
from sklearn.ensemble import RandomForestClassifier
from art.estimators.classification.scikitlearn import ScikitlearnRandomForestClassifier
model = RandomForestClassifier()
model.fit(x_target_train, y_target_train)
art_classifier = ScikitlearnRandomForestClassifier(model)
print('Base model accuracy:', model.score(x_target_test, y_target_test))

Base model accuracy: 0.7534441805225653


In [103]:
# ------------------------------ Shadow Models= 1 ------------------------------------------
from art.attacks.inference.membership_inference import ShadowModels
from art.utils import to_categorical
x_shadow = imgs_test
y_shadow = to_categorical(labels_test)
x_shadow = x_shadow.reshape(x_shadow.shape[0], -1)
y_shadow = to_categorical(np.argmax(y_shadow, axis=1))
shadow_models = ShadowModels(art_classifier, num_shadow_models=1)
shadow_dataset = shadow_models.generate_shadow_dataset(x_shadow, y_shadow)
(member_x, member_y, member_predictions), (nonmember_x, nonmember_y, nonmember_predictions) = shadow_dataset
print([sm.model.score(imgs_val.reshape(imgs_val.shape[0], -1), labels_val) for sm in shadow_models.get_shadow_models()])

[0.5598639455782313]


In [104]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox
attack = MembershipInferenceBlackBox(art_classifier, attack_model_type="rf")
attack.fit(member_x, member_y, nonmember_x, nonmember_y, member_predictions, nonmember_predictions)
member_infer = attack.infer(x_target_train, y_target_train)
nonmember_infer = attack.infer(x_target_test, y_target_test)
member_acc = np.sum(member_infer) / len(x_target_train)
nonmember_acc = 1 - (np.sum(nonmember_infer) / len(x_target_test))
acc = (member_acc * len(x_target_train) + nonmember_acc * len(x_target_test)) / (len(x_target_train) + len(x_target_test))
print('Attack Member Acc:', member_acc)
print('Attack Non-Member Acc:', nonmember_acc)
print('Attack Accuracy:', acc)

Attack Member Acc: 0.919767809419811
Attack Non-Member Acc: 0.8453681710213776
Attack Accuracy: 0.899955723291657


In [93]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox
attack = MembershipInferenceBlackBox(art_classifier, attack_model_type="rf")
attack.fit(member_x, member_y, nonmember_x, nonmember_y, member_predictions, nonmember_predictions)
member_infer = attack.infer(x_target_train, y_target_train)
nonmember_infer = attack.infer(x_target_test, y_target_test)
member_acc = np.sum(member_infer) / len(x_target_train)
nonmember_acc = 1 - (np.sum(nonmember_infer) / len(x_target_test))
acc = (member_acc * len(x_target_train) + nonmember_acc * len(x_target_test)) / (len(x_target_train) + len(x_target_test))
print('Attack Member Acc:', member_acc)
print('Attack Non-Member Acc:', nonmember_acc)
print('Attack Accuracy:', acc)

Attack Member Acc: 0.9341360383919078
Attack Non-Member Acc: 0.792874109263658
Attack Accuracy: 0.8965190073583672


In [94]:
print(calc_precision_recall(np.concatenate((member_infer, nonmember_infer)),
                            np.concatenate((np.ones(len(member_infer)), np.zeros(len(nonmember_infer))))))

(0.9255188907553455, 0.9341360383919078)


In [105]:
# ------------------------------ Shadow Models= 3 ------------------------------------------
from art.attacks.inference.membership_inference import ShadowModels
from art.utils import to_categorical
x_shadow = imgs_test
y_shadow = to_categorical(labels_test)
x_shadow = x_shadow.reshape(x_shadow.shape[0], -1)
y_shadow = to_categorical(np.argmax(y_shadow, axis=1))
shadow_models = ShadowModels(art_classifier, num_shadow_models=3)
shadow_dataset = shadow_models.generate_shadow_dataset(x_shadow, y_shadow)
(member_x, member_y, member_predictions), (nonmember_x, nonmember_y, nonmember_predictions) = shadow_dataset
print([sm.model.score(imgs_val.reshape(imgs_val.shape[0], -1), labels_val) for sm in shadow_models.get_shadow_models()])

[0.5340136054421769, 0.5410430839002267, 0.5446712018140589]


In [96]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox
attack = MembershipInferenceBlackBox(art_classifier, attack_model_type="rf")
attack.fit(member_x, member_y, nonmember_x, nonmember_y, member_predictions, nonmember_predictions)
member_infer = attack.infer(x_target_train, y_target_train)
nonmember_infer = attack.infer(x_target_test, y_target_test)
member_acc = np.sum(member_infer) / len(x_target_train)
nonmember_acc = 1 - np.sum(nonmember_infer) / len(x_target_test)
acc = (member_acc * len(x_target_train) + nonmember_acc * len(x_target_test)) / (len(x_target_train) + len(x_target_test))
print('Attack Member Acc:', member_acc)
print('Attack Non-Member Acc:', nonmember_acc)
print('Attack Accuracy:', acc)

Attack Member Acc: 0.9299117790741114
Attack Non-Member Acc: 0.8047505938242281
Attack Accuracy: 0.8965822597988572


In [97]:
print(calc_precision_recall(np.concatenate((member_infer, nonmember_infer)),
                            np.concatenate((np.ones(len(member_infer)), np.zeros(len(nonmember_infer))))))

(0.9291908344340435, 0.9299117790741114)


In [107]:
# ------------------------------ Shadow Models= 5 ------------------------------------------
from art.attacks.inference.membership_inference import ShadowModels
from art.utils import to_categorical
x_shadow = imgs_test
y_shadow = to_categorical(labels_test)
x_shadow = x_shadow.reshape(x_shadow.shape[0], -1)
y_shadow = to_categorical(np.argmax(y_shadow, axis=1))
shadow_models = ShadowModels(art_classifier, num_shadow_models=5)
shadow_dataset = shadow_models.generate_shadow_dataset(x_shadow, y_shadow)
(member_x, member_y, member_predictions), (nonmember_x, nonmember_y, nonmember_predictions) = shadow_dataset
print([sm.model.score(imgs_val.reshape(imgs_val.shape[0], -1), labels_val) for sm in shadow_models.get_shadow_models()])

[0.5489795918367347, 0.5498866213151927, 0.5458049886621316, 0.5505668934240363, 0.5648526077097505]


In [99]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox
attack = MembershipInferenceBlackBox(art_classifier, attack_model_type="rf")
attack.fit(member_x, member_y, nonmember_x, nonmember_y, member_predictions, nonmember_predictions)
member_infer = attack.infer(x_target_train, y_target_train)
nonmember_infer = attack.infer(x_target_test, y_target_test)
member_acc = np.sum(member_infer) / len(x_target_train)
nonmember_acc = 1 - np.sum(nonmember_infer) / len(x_target_test)
acc = (member_acc * len(x_target_train) + nonmember_acc * len(x_target_test)) / (len(x_target_train) + len(x_target_test))
print('Attack Member Acc:', member_acc)
print('Attack Non-Member Acc:', nonmember_acc)
print('Attack Accuracy:', acc)

Attack Member Acc: 0.9298255697002787
Attack Non-Member Acc: 0.8157561361836896
Attack Accuracy: 0.899449703767737


In [100]:
print(calc_precision_recall(np.concatenate((member_infer, nonmember_infer)),
                            np.concatenate((np.ones(len(member_infer)), np.zeros(len(nonmember_infer))))))

(0.9329085457271364, 0.9298255697002787)
